In [3]:
import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression

# 1) 데이터 로드 (Kaggle Titanic 기준)
## train = pd.read_csv("/kaggle/input/titanic/train.csv")
## test  = pd.read_csv("/kaggle/input/titanic/test.csv")
train = pd.read_csv("./train.csv")
test  = pd.read_csv("./test.csv")

# 2. 데이터 합치기 (전체 통계를 위해)
train_len = len(train)
all_data = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)



In [4]:
# --- [Feature Engineering Start] ---
# (1) Title 추출 및 정리
all_data['Title'] = all_data['Name'].str.extract(r' ([A-Za-z]+)\.', expand=False)
all_data['Title'] = all_data['Title'].replace(['Lady', 'Countess','Capt', 'Col','Don', 
                                               'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
all_data['Title'] = all_data['Title'].replace(['Mlle', 'Ms'], 'Miss')
all_data['Title'] = all_data['Title'].replace('Mme', 'Mrs')

# (2) 나이 채우기 (Title별 중앙값 사용 - 결측치 제거의 정석)
title_ages = all_data.groupby('Title')['Age'].transform('median')
all_data['Age'] = all_data['Age'].fillna(title_ages)

# (3) [핵심] Ticket Frequency (같은 티켓을 가진 사람 수 = 진짜 일행)
all_data['TicketFreq'] = all_data.groupby('Ticket')['Ticket'].transform('count')

# (4) [강제 주입] IsChild & IsAlone (모델이 못 배울까봐 명시적으로 지정)
all_data['IsChild'] = ((all_data['Age'] < 10) | (all_data['Title'] == 'Master')).astype(int)
all_data['IsAlone'] = (all_data['TicketFreq'] == 1).astype(int)

# (5) Fare Binning (돈은 연속적이지 않고 등급이 있음)
all_data['Fare'] = all_data['Fare'].fillna(all_data['Fare'].median())
all_data['FareBin'] = pd.qcut(all_data['Fare'], 4, labels=False) # 4구간으로 나눔

# (6) 불필요 컬럼 Drop
all_data = all_data.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId', 'SibSp', 'Parch'])



In [5]:
# 3. 다시 Train / Test 분리
X = all_data[:train_len].drop(columns=['Survived'])
y = all_data[:train_len]['Survived']
X_test = all_data[train_len:].drop(columns=['Survived'])

# 4. 전처리 파이프라인 (이제 컬럼들이 좀 바뀌었으니 다시 지정)
num_cols = ['Age', 'Fare', 'TicketFreq', 'IsChild', 'IsAlone', 'FareBin']
cat_cols = ['Pclass', 'Sex', 'Embarked', 'Title'] # Pclass는 범주형으로 보는게 유리할 때가 많음

# num_cols = X.select_dtypes(include=["number"]).columns
# cat_cols = X.select_dtypes(exclude=["number"]).columns


In [6]:
# 4) 전처리 파이프(수치형/범주형)
numeric_pipe = Pipeline(steps=[
    ("scaler", StandardScaler()) # 트리는 스케일링 필요 없지만, 습관적으로 해둠
])
# numeric_pipe = Pipeline(steps=[
#     ("imputer", SimpleImputer(strategy="median")),
#     ("scaler", StandardScaler()),
# ])

categorical_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

In [7]:
preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_pipe, num_cols),
        ("cat", categorical_pipe, cat_cols),
    ],
    remainder="drop",  # 지정 안 된 컬럼은 버림
)

In [8]:
# from sklearn.ensemble import GradientBoostingClassifier

# # 모델 변경
# model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

# clf = Pipeline(steps=[
#     ("preprocess", preprocess),
#     ("model", model),
# ])

# 모델 파이프라인 재설정 (앙상블 말고 이것만 쓰세요)
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    n_estimators=300,
    max_depth=8,          # 깊이를 조금 더 줌 (복잡한 상호작용 허용)
    min_samples_split=5,  # 10은 너무 빡빡해서 Master를 무시했을 수 있음
    random_state=42
)

clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", model),
])


In [9]:

# 6) 고정된 CV (StratifiedKFold: 클래스 비율 유지)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(clf, X, y, cv=cv, scoring="accuracy")
print("CV accuracy: %.4f ± %.4f" % (scores.mean(), scores.std()))

CV accuracy: 0.8473 ± 0.0117


In [10]:

# 7) 최종 학습 + 예측 + 제출 파일 생성
clf.fit(X, y)
pred = clf.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": pred.astype(int)
})
submission.to_csv("submission.csv", index=False)
print(submission.head())

   PassengerId  Survived
0          892         0
1          893         0
2          894         0
3          895         0
4          896         1
